# Moss Tool Example

This notebook demonstrates how to use the `MossToolSpec` with LlamaIndex.

## Installation

First, you need to install the package (and LlamaIndex core if not present).

In [ ]:
%pip install llama-index-tools-moss llama-index-core llama-index-llms-openai

## Setup

Import the necessary classes and initialize the Moss client.

In [ ]:
import os
from llama_index.tools.moss import MossToolSpec, QueryOptions
from inferedge_moss import MossClient, DocumentInfo

# Initialize Moss Client
MOSS_PROJECT_KEY = os.getenv('MOSS_PROJECT_KEY')
MOSS_PROJECT_ID = os.getenv('MOSS_PROJECT_ID')
client = MossClient(project_id=MOSS_PROJECT_ID, project_key=MOSS_PROJECT_KEY)

# 2. Configure query settings - Instantiate QueryOptions (Optional)
# If skipped, the tool will use its own defaults.
query_options = QueryOptions(top_k=12, alpha=0.9)
# 3. Initialize Tool
print("Initializing MossToolSpec...")
moss_tool = MossToolSpec(
    client=client,
    index_name="knowledge_base",
    query_options=query_options
)


## Indexing Data

Let's add some sample data to our index.

In [ ]:
docs = [
    DocumentInfo(
        text="LlamaIndex connects your data to LLMs.", 
        metadata={"topic": "AI"}
    ),
    DocumentInfo(
        text="Moss provides fast semantic search integration.", 
        metadata={"topic": "Search"}
    ),
]

# Index the documents
await moss_tool.index_docs(docs)

## Using with an Agent

Now we can wrap the tool for use with a LlamaIndex agent.

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

# Convert to tool list
tools = moss_tool.to_tool_list()

# Create an agent (Using OpenAI llm for demonstration)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-here')
llm = OpenAI()
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

# Chat with the agent
response = await agent.achat("What does Moss provide?")
print(response)